In [ ]:
!kaggle datasets download -d shubhamgoel27/dermnet -p /content/drive/MyDrive --unzip

Dataset URL: https://www.kaggle.com/datasets/shubhamgoel27/dermnet
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 99% 1.71G/1.72G [00:24<00:00, 46.5MB/s]
100% 1.72G/1.72G [00:24<00:00, 76.7MB/s]


In [ ]:
trainpath = "/content/drive/MyDrive/train"
testpath = "/content/drive/MyDrive/test"

In [ ]:


'''import shutil
shutil.rmtree(trainpath, ignore_errors=True)'''


In [ ]:
import os

class_image_counts = {class_folder: len([f for f in os.listdir(os.path.join(trainpath, class_folder)) if f.endswith('.jpg')])
                      for class_folder in os.listdir(trainpath) if os.path.isdir(os.path.join(trainpath, class_folder))}

for class_name, count in class_image_counts.items():
    print(f" '{class_name}'  -  {count}  images.")

 'Acne and Rosacea Photos'  -  840  images.
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions'  -  1149  images.
 'Atopic Dermatitis Photos'  -  489  images.
 'Bullous Disease Photos'  -  448  images.
 'Cellulitis Impetigo and other Bacterial Infections'  -  288  images.
 'Eczema Photos'  -  1235  images.
 'Exanthems and Drug Eruptions'  -  404  images.
 'Hair Loss Photos Alopecia and other Hair Diseases'  -  239  images.
 'Herpes HPV and other STDs Photos'  -  405  images.
 'Light Diseases and Disorders of Pigmentation'  -  568  images.
 'Lupus and other Connective Tissue diseases'  -  420  images.
 'Melanoma Skin Cancer Nevi and Moles'  -  463  images.
 'Nail Fungus and other Nail Disease'  -  1040  images.
 'Poison Ivy Photos and other Contact Dermatitis'  -  260  images.
 'Psoriasis pictures Lichen Planus and related diseases'  -  1405  images.
 'Scabies Lyme Disease and other Infestations and Bites'  -  431  images.
 'Seborrheic Keratoses and other Benign Tumors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 49.2 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from skimage.restoration import denoise_wavelet
from skimage.util import img_as_ubyte
from tqdm import tqdm
import os

def rs(img, sz=(224, 224)):
    return cv2.resize(img, sz, interpolation=cv2.INTER_AREA)

def nm(img):
    return img.astype(np.float32) / 255.0

def rh(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    bh = cv2.morphologyEx(g, cv2.MORPH_BLACKHAT, k)
    _, th = cv2.threshold(bh, 10, 255, cv2.THRESH_BINARY)
    return cv2.inpaint(img, th, inpaintRadius=1, flags=cv2.INPAINT_TELEA)

def dn(img):
    ch = [denoise_wavelet(img[:, :, i], method='BayesShrink', mode='soft', wavelet_levels=1, rescale_sigma=True)
          for i in range(img.shape[2])]
    return img_as_ubyte(np.stack(ch, axis=-1))

def pp(img_p, d=False):
    img = cv2.imread(img_p)
    if img is None:
        print(f"Err: {img_p} not found")
        return None
    img = rh(img)
    if d:
        img = dn(img)
    return img

def pp_ds(tr_p, out_p, d=False):
    if not os.path.exists(out_p):
        os.makedirs(out_p)
    for sb in os.listdir(tr_p):
        sb_p = os.path.join(tr_p, sb)
        out_sb_p = os.path.join(out_p, sb)
        if os.path.isdir(sb_p):
            if not os.path.exists(out_sb_p):
                os.makedirs(out_sb_p)
            for img_n in tqdm(os.listdir(sb_p), desc=f"Processing {sb}"):
                img_p = os.path.join(sb_p, img_n)
                p_img = pp(img_p, d)
                if p_img is not None:
                    cv2.imwrite(os.path.join(out_sb_p, img_n), p_img)

tr_p = trainpath
out_p = "/content/drive/MyDrive/preprocessed"
pp_ds(tr_p, out_p, d=True)


Processing Acne and Rosacea Photos: 100%|██████████| 840/840 [02:32<00:00,  5.49it/s]
Processing Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions: 100%|██████████| 1149/1149 [03:36<00:00,  5.30it/s]
Processing Bullous Disease Photos: 100%|██████████| 448/448 [01:17<00:00,  5.78it/s]
Processing Cellulitis Impetigo and other Bacterial Infections: 100%|██████████| 288/288 [00:56<00:00,  5.13it/s]
Processing Exanthems and Drug Eruptions: 100%|██████████| 404/404 [01:12<00:00,  5.57it/s]
Processing Hair Loss Photos Alopecia and other Hair Diseases: 100%|██████████| 239/239 [00:48<00:00,  4.92it/s]
Processing Herpes HPV and other STDs Photos: 100%|██████████| 405/405 [01:18<00:00,  5.15it/s]
Processing Light Diseases and Disorders of Pigmentation: 100%|██████████| 568/568 [01:40<00:00,  5.67it/s]
Processing Lupus and other Connective Tissue diseases: 100%|██████████| 420/420 [01:18<00:00,  5.33it/s]
Processing Nail Fungus and other Nail Disease: 100%|██████████| 1040/1040 [

In [ ]:
from google.colab import files
import shutil
folder_path = "/content/drive/MyDrive/preprocessed"
shutil.make_archive("preprocessed_images", 'zip', folder_path)

files.download("preprocessed_images.zip")


In [1]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


/Users/geetika/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Num GPUs Available: 1
